# College Basketball Code
## Dan McDonough
## February 25th, 2021

The purpose of this notebook is to compile four factors stats for D1 NCAAM teams ahead of march madness. My goal is to write functionality that gives a very condensed, stylistic game preview to digest ahead of watching the game.

### 1. Scrape Data

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import random
import timeit
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import scipy as sp

I've scraped the data from sports reference, which maintains tables of advanced stats for each team

In [2]:
# url = 'https://www.sports-reference.com/cbb/seasons/2022-advanced-school-stats.html'
# response = requests.get(url)
# soup = BeautifulSoup(response.text, "html.parser")

# datarows = soup.findAll('tr')

# team = []
# srs = []
# sos = []
# pace = []
# ortg = []
# ftr = []
# efg = []
# tov = []
# orb = []

# for i in range(2,len(datarows)):
#     if i in [22,23,44,45,66,67,88,89,110,111,132,133,154,155,176,177,198,199,220,221,242,243,264,265,286,287,308,309,330,331,
#             352,353,374,375]:
#         continue
#     team.append(str(datarows[i].findAll('td')[0].findAll('a')).split('>')[1].split('<')[0])
#     srs.append(str(datarows[i].findAll('td')[5]).split('>')[1].split('<')[0])
#     sos.append(str(datarows[i].findAll('td')[6]).split('>')[1].split('<')[0])
#     pace.append(str(datarows[i].findAll('td')[20]).split('>')[1].split('<')[0])
#     ortg.append(str(datarows[i].findAll('td')[21]).split('>')[1].split('<')[0])
#     ftr.append(str(datarows[i].findAll('td')[22]).split('>')[1].split('<')[0])
#     efg.append(str(datarows[i].findAll('td')[29]).split('>')[1].split('<')[0])
#     tov.append(str(datarows[i].findAll('td')[30]).split('>')[1].split('<')[0])
#     orb.append(str(datarows[i].findAll('td')[31]).split('>')[1].split('<')[0])

# stats_df = pd.DataFrame({'Team':team,'SRS':srs,'SOS':sos,'Pace':pace,'OFF':ortg,'FTR':ftr,'EFG':efg,'TOV':tov,
#                           'ORB':orb})

In [3]:
# stats_df.head(2)

In [4]:
# url = 'https://www.sports-reference.com/cbb/seasons/2022-advanced-opponent-stats.html'
# response = requests.get(url)
# soup = BeautifulSoup(response.text, "html.parser")

# datarows = soup.findAll('tr')

# team = []
# srs = []
# sos = []
# pace = []
# drtg = []
# oppftr = []
# oppefg = []
# opptov = []
# opporb = []

# for i in range(2,len(datarows)):
# #     if ((i == 22) | (i == 33)):
#     if i in [22,23,44,45,66,67,88,89,110,111,132,133,154,155,176,177,198,199,220,221,242,243,264,265,286,287,308,309,330,331,
#             352,353,374,375]:
#         continue
#     team.append(str(datarows[i].findAll('td')[0].findAll('a')).split('>')[1].split('<')[0])
#     srs.append(str(datarows[i].findAll('td')[5]).split('>')[1].split('<')[0])
#     sos.append(str(datarows[i].findAll('td')[6]).split('>')[1].split('<')[0])
#     pace.append(str(datarows[i].findAll('td')[20]).split('>')[1].split('<')[0])
#     drtg.append(str(datarows[i].findAll('td')[21]).split('>')[1].split('<')[0])
#     oppftr.append(str(datarows[i].findAll('td')[22]).split('>')[1].split('<')[0])
#     oppefg.append(str(datarows[i].findAll('td')[29]).split('>')[1].split('<')[0])
#     opptov.append(str(datarows[i].findAll('td')[30]).split('>')[1].split('<')[0])
#     opporb.append(str(datarows[i].findAll('td')[31]).split('>')[1].split('<')[0])

# opp_stats_df = pd.DataFrame({'Team':team,'SRS':srs,'SOS':sos,'Pace':pace,'DEF':drtg,'Opp FTR':oppftr,'Opp EFG':oppefg,
#                          'Opp TOV':opptov,'Opp ORB':opporb})

In [5]:
# opp_stats_df.tail(2)

In [6]:
# combined_df = stats_df.merge(opp_stats_df[['Team','DEF','Opp FTR','Opp EFG','Opp TOV','Opp ORB']],on='Team')

Save file for future use

In [7]:
# combined_df.to_csv('college.csv',index=False)

In [8]:
combined_df = pd.read_csv('college.csv')

combined_df['FTR'] = combined_df['FTR'] * 100
combined_df['EFG'] = combined_df['EFG'] * 100
combined_df['Opp FTR'] = combined_df['Opp FTR'] * 100
combined_df['Opp EFG'] = combined_df['Opp EFG'] * 100

### 2. Write Summary Function

In [24]:
def game_summary(away,home):
    away_df = combined_df[combined_df['Team']==away]
    home_df = combined_df[combined_df['Team']==home]
    home_df_adj = home_df[['Team', 'SRS', 'SOS', 'Pace','DEF', 'Opp FTR', 'Opp EFG', 'Opp TOV', 'Opp ORB', 'OFF', 'FTR', 
                           'EFG', 'TOV', 'ORB']].copy()
    away_df.columns = ['Team', 'SRS', 'SOS', 'Pace','Away Rtg', 'Away FTR', 'Away EFG', 'Away TOV', 'Away ORB',
                       'Home Rtg', 'Home FTR', 'Home EFG', 'Home TOV', 'Home ORB']
    home_df_adj.columns = ['Team', 'SRS', 'SOS', 'Pace','Away Rtg', 'Away FTR', 'Away EFG', 'Away TOV', 'Away ORB',
                       'Home Rtg', 'Home FTR', 'Home EFG', 'Home TOV', 'Home ORB']
    output_df = pd.concat([away_df,home_df_adj],axis=0)
    means_df = pd.DataFrame(output_df.mean()).transpose()
    output_df = pd.concat([output_df,means_df],sort=False).reset_index(drop=True)
    output_df.loc[2,'Team']='Average'
    for column in output_df.columns[1:]:
        output_df[column] = output_df[column].astype(int)
    return output_df

In [25]:
game_summary('Iowa','Nebraska')

,Team,SRS,SOS,Pace,Away Rtg,Away FTR,Away EFG,Away TOV,Away ORB,Home Rtg,Home FTR,Home EFG,Home TOV,Home ORB
0,Iowa,18,6,70,116,30,52,10,31,99,27,49,17,29
1,Nebraska,1,8,72,106,28,51,16,31,97,33,49,15,20
2,Average,10,7,71,111,29,52,13,31,98,30,49,16,25


In [26]:
game_summary('Texas State','Troy')

,Team,SRS,SOS,Pace,Away Rtg,Away FTR,Away EFG,Away TOV,Away ORB,Home Rtg,Home FTR,Home EFG,Home TOV,Home ORB
0,Texas State,0,-2,66,107,31,52,16,31,94,33,48,19,26
1,Troy,-2,-3,69,92,32,46,19,26,99,32,49,19,31
2,Average,0,-3,68,99,31,49,17,28,96,33,48,19,28
